#Загрузка библиотек


In [ ]:
!pip install ultralytics

In [ ]:
from google.colab import drive # Для работы с Google Drive
drive.mount('/content/drive/')
directory = '/content/drive/My Drive/ColabData/'

#Рабочий код для локальной работы

In [ ]:
from ultralytics import YOLO
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image
from collections import defaultdict
import numpy as np


centr=np.array((1024,768)) # Центральная точка кадра

track_history = defaultdict(lambda: [])  # история трекинга
directory = 'C:/users/Home/Documents/Kaski/' # рабочая директория
save_dir=directory+'violators/'  # директория для записи кадров с нарушителями
test_path = directory+'576' # директория, откуда берутся картинки для распознавания (нарезанные из видео)

model = YOLO(directory+'best.pt') # Загружаем обученную модель


for image in os.listdir(test_path)[:100:1]: # диапазон картинок для распознавания
  img_path = os.path.join(test_path, image)

  results = model.track(img_path, classes=1, persist=True)  # classes=1 - No Hardhat,
  persons_count = results[0].__len__()  # number of No Hardhat detected
  #print('Persons=', persons_count)#для отладки
  annotated_frame = results[0].plot()
  for result in results:
      if result.boxes.id is None:  # Expecting to skip this iteration if no objects are detected.
          continue

      boxes = results[0].boxes.xywh.cpu()
      track_ids = results[0].boxes.id.int().cpu().tolist()

       # Разбираем треки
      for track_id, box in zip(track_ids, boxes):
          x, y, w, h = box
          track = track_history[track_id]
          track.append((int(x), int(y)))  # x, y center point

          #print('Center=', int(x), int(y),'   id=',track_id) # для отладки

          for i in track_history: # проверяем все ID
            if len(track_history[i])==1: # если это первый кадр для очередного ID, то записываем его
                cv2.imwrite(f'{save_dir} {i:03d}.jpg', annotated_frame)
            else:             #если расстояние от центра box до центра кадра меньше, чем предыдущее, то записываем кадр
                if np.linalg.norm(np.array(track_history[i][1])-centr) < np.linalg.norm(np.array(track_history[i][0])-centr):
                    cv2.imwrite(f'{save_dir} {i:03d}.jpg', annotated_frame)

          if len(track) > 2:  # максимальная длина списка (трека). Сейчас сохраняются только 2 значения
              track.pop(0)

         # Рисование линии трека (сейчас только 2 точки)
      points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
      cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=5)

      # отображение на мониторе -------------------------
  annotated_frame=cv2.resize(annotated_frame, (640,480), cv2.INTER_NEAREST)
  cv2.imshow("YOLOv8 Inference", annotated_frame)
  cv2.waitKey(500)
